# Pipelining with Titanic Data

### Data loading ...

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
combined = pd.concat([train, test]).reset_index(drop=True)

In [ ]:
combined.Cabin.value_counts()

#### Some preprocessing of the data

In [ ]:
def parse_name(s): 
    a, b = s.split(',')
    family_name = a.strip()
    title = b.split('.')[0].strip()
    first_name = b.split('.')[1].split()[0].strip()
    return (first_name.replace('(', '').replace(')', ''), title, family_name)

def parse_cabin_letter(column):
    counts = column.value_counts()
    letter_pat = re.compile('([A-Za-z])\d?')
    return list(map(lambda x: letter_pat.findall(x)[0] if letter_pat.findall(x) else pd.np.nan, column.values.tolist()))

def parse_ticket_number(column):
    number_pat = re.compile('\d{3,}')
    numbers = map(lambda x: number_pat.findall(x)[0] if number_pat.findall(x) else pd.np.nan, column)
    return pd.Series(numbers)


def get_friendship_group(df):
    friendship_group_counter = 0
    if 'family_name' not in df.columns:
        family_names = pd.Series(map(lambda x: parse_name(x)[-1], df.Name))
    else:
        family_names = df.family_name
    cabins = pd.Series(map(parse_cabin_letter, df.Cabin))
    ticket_grouping = []
    for family, count in family_names.value_counts().items():
        family_mask = family_names == family
        if count == 1:
            ticket_grouping.append(friendship_group_counter)
            friendship_group_counter += 1
            continue
        
        
    
    
    
        


In [ ]:
grouped = combined.Ticket.groupby(combined.Ticket)
grouped.apply(lambda x: print(x.iloc[0]))

## Utility functions and classes

In [ ]:
# self explanatory function, returns first name, title and the last name of a passenger
def parse_name(s): 
    a, b = s.split(',')
    family_name = a.strip()
    title = b.split('.')[0].strip()
    first_name = b.split('.')[1].split()[0].strip()
    return (first_name, title, family_name)

def parse_cabin(s):
    s.split()
    


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from IPython.core.display import display, HTML

class AbstractPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise Exception("Input data should be a DataFrame instance")
        return self


class DropColumns(AbstractPreprocessor):
    '''
    Drops specified columns from a DataFrame.
    
    Input data assumed to be a DataFrame.
    
    Usage
    -----
        dropper = DropColumns(names=['PassengerId', 'Ticket'])
        dropped_df = dropper.fit_transform(source_dataframe)
    '''
    
    def __init__(self, names=[]):
        self.names = names
    
    def transform(self, X, y=None):
        filtered = [col for col in self.names if col in X.columns]
        return X.drop(filtered, axis=1)

class AddFirstNameColumn(AbstractPreprocessor):

    def transform(self, X, y=None):
        result = X.copy()
        result['first_name'] = X.loc[:, 'Name'].apply(lambda x: parse_name(x)[0].replace('(', '').replace(')', ''))
        return result

class AddFamilyNameColumn(AbstractPreprocessor):
    def transform(self, X, y=None):
        result = X.copy()
        result['family_name'] = X.loc[:, 'Name'].apply(lambda x: parse_name(x)[-1])
        return result
                
class AddTitleColumn(AbstractPreprocessor):

    def transform(self, X, y=None):
        result = X.copy()
        result['title'] = X.loc[:, 'Name'].apply(lambda x: parse_name(x)[1])
        return result   

class AddFamilySize(AbstractPreprocessor):
    def transform(self, X, y=None):
        _X = X.copy()
        _X['family_size'] = X['SibSp'] + X['Parch'] + 1
        return _X

class AddIsAlone(AbstractPreprocessor):
    def transform(self, X, y=None):
        _X = X.copy()
        _X['is_alone'] = 1
        _X.loc[_X.family_size > 1,'is_alone'] = 0
        return _X

class OneHotEncodeAndDrop(AbstractPreprocessor):
    
    def __init__(self, name=None):
        self.name = name
    
    def transform(self, X, y=None):
        aux = pd.get_dummies(X.loc[:, self.name], prefix=self.name)
        return pd.concat([X.drop(self.name, axis=1), aux], axis=1)

class LabelEncodeAndDrop(AbstractPreprocessor):
    
    def __init__(self, name=None):
        self.name = name
        self.enc = LabelEncoder()
    
    def fit(self, X, y=None):
        self.enc.fit(X.loc[:, self.name].values)
        return self
        
    def transform(self, X, y=None):
        _X = X.copy()
        new_values = self.enc.transform(_X.loc[:, self.name].values)
        _X.loc[:, self.name] = new_values
        return _X

class SelectFeatures(AbstractPreprocessor):
    
    def __init__(self, k, n):
        self.k = k
        self.n = n

    def transform(self, X, y=None):
        _ = [int(x) for x in bin(self.k)[2:]]
        _ = [0] * (self.n - len(_)) + _
        return X.iloc[:, [j for j in range(self.n) if _[j]]]
    
    
class CombineCategoricalValues(AbstractPreprocessor):
    def __init__(self, name=None, rule={'what': [], 'to': None}):
        self.name = name
        self.rule = rule
        
    def transform(self, X, y=None):
        mask = X.loc[:, self.name].isin(self.rule['what'])
        _X = X.copy()
        _X.loc[:, self.name][mask] = self.rule['to']
        return _X
        
class DropByValue(AbstractPreprocessor):
    def __init__(self, name=None, value=None):
        self.name = name
        self.value = value
    
    def transform(self, X, y=None):
        return X[X.loc[:, self.name] != self.value]
    
class GetCategoriesAndDrop(AbstractPreprocessor):
    def __init__(self, name=None, bins=None):
        self.name = name
        self.bins = bins
    
    def transform(self, X, y=None):
        if self.bins is not None and hasattr(X, self.name):
            aux = pd.qcut(X.loc[:, self.name], q=self.bins, labels=False)
            _X = X.drop(self.name, axis=1)
            _X[self.name] = aux
            return _X
        else:
            return X
   
    
from sklearn.ensemble import RandomForestRegressor
class FillNaValues(AbstractPreprocessor):

    def __init__(self, name=None, train=None, n_features=None,
                 clf=RandomForestRegressor()):
        self.train = train
        self.name = name
        self.clf = clf
        self.n_features = n_features
    
    def transform(self, X, y=None):

        if self.name is None: 
            return X

        if X.loc[:, self.name].isnull().sum() == 0:
            return X
        
        _train = self.train.copy() if self.train is not None else X.copy()
        null_mask = _train[self.name].isnull()
        y = _train[self.name][~null_mask]
        _train = _train.drop(self.name, axis=1)
        
        n_features = int(pd.np.ceil(X.shape[1] * 0.3) or self.n_features)
        
        encoders = dict()
        for key in _train.columns.tolist():
            if not pd.np.issubdtype(_train[key].dtype, pd.np.number):
                _train.loc[_train[key].isnull(), key]  = 'N-a-N'
                le = LabelEncoder()
                _train[key] = le.fit_transform(_train[key])
                encoders[key] = le
            else:
                if any(_train[key].isnull()):
                    _train['%s_nan' % key] = 0.0
                    _train.loc[_train[key].isnull(), '%s_nan' % key] = 1.0
                    _train.loc[_train[key].isnull(), key] = _train.loc[~_train[key].isnull(), key].median()

        self.clf.fit(_train[~null_mask], y)
        
        # dropping features
        if hasattr(self.clf, 'feature_importances_'):
            # drop columns and retrain classifier
            indices = pd.np.argsort(self.clf.feature_importances_)[::-1]
            features_to_drop = _train.columns[indices].values.tolist()[n_features:]
            self.clf.fit(_train.drop(features_to_drop, axis=1)[~null_mask], y)
        else:
            features_to_drop = []
            
        _X = X.copy()
        for key in _train.columns:
            if key not in _X.columns:
                _X.loc[:, key] = 0.0
        _X = _X[_train.columns]
        for key in encoders.keys():
            if not pd.np.issubdtype(_X[key].dtype, pd.np.number):
                _X.loc[_X[key].isnull(), key]  = 'N-a-N'
                _X[key] = encoders[key].transform(_X[key])
            else:
                if any(_X[key].isnull()):
                    _X['%s_nan' % key] = 0.0
                    _X.loc[_X[key].isnull(), '%s_nan' % key] = 1.0
                    _X.loc[_X[key].isnull(), key] = X.loc[~_X[key].isnull(), key].median()
        
        na_replacements = self.clf.predict(_X.drop(features_to_drop, axis=1)[null_mask])
        result = X.copy()
        result.loc[null_mask, self.name] = na_replacements
        return result


class FillNaSimple(AbstractPreprocessor):
    def __init__(self, name):
        self.name = name
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        _X = X.copy()
        if hasattr(_X, self.name):
            _X.loc[:, self.name] = _X.loc[:, self.name].fillna(_X.loc[:, self.name].dropna().mode()[0])
        return _X
    
class DropRows(AbstractPreprocessor):

    def __init__(self, condition=None):
        self.condition = condition #condition depends on (X, y) and 
        # returns boolean array of the same length as X

    def fit(self, X, y=None):
        return self
    
    def trasform(self, X, y=None):
        if self.condition is not None:
            return X[self.condition(X, y)]
        else:
            return X


class ShowDataHead(AbstractPreprocessor):
    def fit(self, X, y=None):
        display(HTML(X.head().to_html()))
        return self
        
    def transform(self, X, y=None):
        return X

# Preprocessing steps (feature engeneering)

In [ ]:
from sklearn.pipeline import Pipeline
preprocessing_steps = [('drop_columns', DropColumns(names=['Ticket', 'PassengerId', 'Cabin', 'Survived'])),
                       ('add_title', AddTitleColumn()),
                       # ('add_first_name', AddFirstNameColumn()),
                       #('add_family_name', AddFamilyNameColumn()),
                       ('add_family_size', AddFamilySize()),
                       ('encode_sex', LabelEncodeAndDrop(name='Sex')),
                       ('fillna_embarked', FillNaSimple(name='Embarked')),
                       #('combine_embarked', CombineCategoricalValues(name='Embarked', rule={'what': ['C', 'Q'], 'to': 'Q'})),
                       ('encode_embarked', LabelEncodeAndDrop(name='Embarked')),
                       
                       # predicting nan-values
                       ('predict_fare', FillNaSimple(name='Fare')),
                       ('predict_age', FillNaSimple(name='Age')),
                    #  ('cat_ages', GetCategoriesAndDrop(name='Age', bins=[0, 0.3, 0.6, 1.0])),
                    #  ('cat_fares', GetCategoriesAndDrop(name='Fare', bins=[0, 0.3, 0.6, 1.0])),
                       
                       # combine & encode titles
                       #('combine_title_Mrs', CombineCategoricalValues(name='title', rule={'what': ['Dona', 'Mme', 'the Countess'], 'to': 'Mrs'})),
                       #('combine_title_Mr', CombineCategoricalValues(name='title', rule={'what': ['Jonkheer', 'Major', 'Col', 'Rev', 'Mr'], 'to': 'Mr1'})),
                       #('combine_title_Mr', CombineCategoricalValues(name='title', rule={'what': ['Master', 'Sir', 'Don', 'Dr'], 'to': 'Mr2'})),
                       #('combine_title_Miss', CombineCategoricalValues(name='title', rule={'what': ['Mlle','Lady','Ms'], 'to': 'Miss'})),
                       ('combine_title_rare', CombineCategoricalValues(name='title', rule={'what': ['Rev', 'Dr', 'Col', 'Ms', 'Mlle', 'Major', 'Sir', 'Mme','Lady','Capt','the Countess','Jonkheer','Don','Dona'], 'to': 'rare'})),
                       ('encode_title', LabelEncodeAndDrop(name='title')),
                       
                       #
                       #('add_is_alone', AddIsAlone()),
                       
                       # last one-hots... 
                       #('encode_fare', OneHotEncodeAndDrop(name='Fare')),
                       #('encode_age', OneHotEncodeAndDrop(name='Age')),
                       #('encode_Pclass', OneHotEncodeAndDrop(name='Pclass')),
                       #('combine_fsize', CombineCategoricalValues(name='family_size', rule={'what': ['5','6', '7','8','9','10','11'], 'to': '5'})),
                       #('encode_fsize', OneHotEncodeAndDrop(name='family_size')),
                       
                       #drop first name, family name
                       ('drop_last', DropColumns(names=['first_name','family_name', 'Name'])),
                       ('show_data', ShowDataHead())
                      ]

preprocessing_pipeline = Pipeline(steps=preprocessing_steps)
combined_processed = preprocessing_pipeline.fit_transform(combined)